In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git 
!pip install -q git+https://github.com/huggingface/peft.git #peft 에러 때문에 이 버전 써야 함
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

# 데이터셋 정리

데이터는 csv 형태로 저장했다.

In [2]:
import pandas as pd
data = pd.read_csv('train.csv')

데이터셋의 구조는 다음과 같다. \
Q는 질문이고, A는 대답, label은 중립, 정보있음, 말장난의 세 분류를 각 0, 1, 2로 두었다.

In [3]:
data.head()

,Q,A,label
0,기업은행 신입행원 떨어지고,저도 똑같아요,0
1,두번 다시 하기 싫은 헤어짐,헤어짐는 순간은 악몽과 같아요.,1
2,15 여 경제융소 IT SW 직군 취업후기,수업 때 뵌 것 같네요. 축하드립니다.,0
3,정제천이 서강을 구했다!!,5000명 이전근거는 남양주시가 국토부에 제출한 문건입니다이 문건을 남양주 TFT에...,0
4,행시준비생 금융 공기업 준비 조언 부탁드려요,저도 cpa하다가 알아보는데 취준 친구 만나서 묻는게 젤빠르더라구요.,0


In [ ]:
data.iloc[1]

Q          두번 다시 하기 싫은 헤어짐
A        헤어짐는 순간은 악몽과 같아요.
label                    1
Name: 1, dtype: object

각 데이터는 아래와 같은 방법으로 불러올 수 있다.

In [ ]:
data.iloc[1]['Q']

'두번 다시 하기 싫은 헤어짐'

Koalpaca를 이용하여 해당 데이터를 파인튜닝 하기 위해 데이터셋 스타일을 맞춰주자.

In [4]:
import datasets
data =  datasets.dataset_dict.Dataset.from_dict(data)

In [5]:
data = data.map(
    lambda x: {'text': f"### 질문: {x['Q']}\n\n### 답변: {x['A']}<|endoftext|>" }
)
data = data.rename_column('Q', 'instruction')
data = data.rename_column('A', 'output')
data = data.rename_column('label',  'url')

Map:   0%|          | 0/45096 [00:00<?, ? examples/s]

아래 양식대로 생성한다.

## 쪼개진 모델 로드

- 원래는 단일 파일이기도 하지만, 작은 파일(약 1GB)로 쪼개서 개별로 로드한 레포를 쓰면 RAM이 터지지 않습니다.
- 대신 시간이 오래 걸림... 속이 터집니다.

In [51]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# model_id = "EleutherAI/polyglot-ko-12.8b"
model_id = "beomi/KoAlpaca-Polyglot-12.8B"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

Loading checkpoint shards:   0%|          | 0/28 [00:00<?, ?it/s]

text 데이터의 tokenize

In [52]:

data = data.map(lambda samples: tokenizer(samples["text"]), batched=True)
data

Map:   0%|          | 0/45096 [00:00<?, ? examples/s]

Dataset({
    features: ['instruction', 'output', 'url', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 45096
})

In [53]:
data = data.rename_column('label',  'url')

ValueError: Original column name label not in the dataset. Current columns in the dataset: ['instruction', 'output', 'url', 'text', 'input_ids', 'token_type_ids', 'attention_mask']

PEFT를 통해 `prepare_model_for_kbit_training`로 Low bit 학습을 준비해줍시다.

In [54]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [55]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8, 
    lora_alpha=32, 
    target_modules=["query_key_value"], 
    lora_dropout=0.05, 
    bias="none", 
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 6553600 || all params: 6607912960 || trainable%: 0.09917806181272702


# 학습

In [67]:
import transformers

# needed for gpt-neo-x tokenizer
tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=data,

    args=transformers.TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=1,
        max_steps=45000, ## 초소량만 학습: 1k step만 학습. 약 23분정도 걸립니다.
        learning_rate=1e-4,
        fp16=True,
        logging_steps=10,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
model.padding = True,
model.truncation=True,
trainer.train()

/home/kjh/anaconda3/envs/koalpaca/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
10,5.180900
20,5.132800
30,5.257800
40,5.277300
50,5.173800
60,5.203500
70,5.116000
80,5.382000
90,5.492200
100,5.374600


TrainOutput(global_step=45000, training_loss=5.266590104166666, metrics={'train_runtime': 63220.9809, 'train_samples_per_second': 2.847, 'train_steps_per_second': 0.712, 'total_flos': 3.132864937003008e+17, 'train_loss': 5.266590104166666, 'epoch': 3.99})

In [4]:
model.eval()
model.config.use_cache = True  # silence the warnings. Please re-enable for inference!

NameError: name 'model' is not defined

In [58]:
input_tokens = tokenizer("### 질문: 등록금 납부하려면 어디로 가야 해?", return_tensors='pt')
input_ids = input_tokens.input_ids.to(0)
input_ids

tensor([[    6,     6,     6,  2438,    29,  7586,  6472,   284,  2604,  2116,
           286, 15983,   547,    34]], device='cuda:0')

In [60]:
model.generate(input_ids[0])

TypeError: PeftModelForCausalLM.generate() takes 1 positional argument but 2 were given

In [2]:
def gen(x):
    gened = model.generate(
        **tokenizer(
            f"### 질문: {x}\n\n### 답변:", 
            return_tensors='pt', 
            return_token_type_ids=False
        ).to(0), 
        max_new_tokens=256,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
    )
    print(tokenizer.decode(gened[0]))

In [3]:
gen('건강하게 살기 위한 세 가지 방법은?')

NameError: name 'model' is not defined

In [63]:
gen('서강대학교 등록금 납부하려면 어디로 가야 해?')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 서강대학교 등록금 납부하려면 어디로 가야 해?

### 답변:1. 고지서 출력: 고지서를 인쇄해야 합니다.(우선, 고지서는 학교 홈페이지에서 출력하세요.)
2. 등록금 고지서 납부: 고지서를 가지고 은행에 가서 등록금를 납부해야 합니다.
3. 등록금 완납: 등록금 고지서를 은행에 제출하면, 대학본부, 등록처에서 등록이 완료됩니다. 그 후 '등록 완료되었다'는 연락을 받게 됩니다.(등록이 완료되면 학생증 발급이 가능합니다.)<|endoftext|>


In [1]:
gen('익스트림 수요미식회 빌런 찾습니다')

NameError: name 'gen' is not defined

In [65]:
gen('주변에 7학기하고 휴학하시는 분들 많나요?')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 주변에 7학기하고 휴학하시는 분들 많나요?

### 답변:네, 많이 있습니다. 7학기째에 휴학하는 것은 일반적인 것 같습니다. 군복무를 위한 경우도 있고, 본인의 미래를 위한 계획을 세우기 위해, 그리고 다양한 이유로 인해 휴학을 하는 경우가 있습니다. 7학기째는 일반적으로 휴학을 많이 하는 학기입니다. 그러나 학교마다 다르기 때문에, 학교 내에서 다양한 이유로 인해 휴학하는 분들을 많이 만날 수 있습니다. <|endoftext|>


In [66]:
gen('바닥부터 영상 배워보고 싶은데 현실적인 조언 부탁드립니다.,')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 바닥부터 영상 배워보고 싶은데 현실적인 조언 부탁드립니다.,

### 답변:영상 제작 분야에서 바닥부터 시작하는 것은 매우 어려운 일입니다. 하지만, 꼭 영상 제작 분야에서 성공하고 싶다면, 천천히 가는 것도 좋은 선택입니다. 먼저, 영상 제작과 관련된 기술 및 지식을 배우기 위해 학원에 다니며 기초를 탄탄히 다지는 것이 좋습니다. 기술적인 지식 뿐만 아니라, 기획, 촬영, 편집 등 영상 제작 프로세스 전반적인 것을 배울 수 있기 때문입니다. 더불어, 책이나 강의를 통해 본인이 원하는 분야에 대해 집중적으로 공부하세요. 이와 함께, 다양한 영상 제작 경험을 쌓을 수 있는 아르바이트나 직장을 찾아보시는 것도 좋은 방법입니다. 꾸준한 노력과 인내심을 가지고 이 길을 걸어가다 보면, 좋은 날이 올 것입니다. <|endoftext|>
